In [1]:
import time
import heapq
from collections import deque

# Shared Environment Generation
def generate_environment(rows=5, cols=6):
    room = [["Clean" for _ in range(cols)] for _ in range(rows)]
    dirty_positions = [(0, 1), (0, 4), (1, 1), (2, 3), (3, 0), (4, 2)]
    for r, c in dirty_positions:
        room[r][c] = "Dirty"
    return room

# 1. DLS and IDS
def dls_vacuum(room, start_pos, limit):
    rows = len(room)
    cols = len(room[0])

    initial_dirty = set(
        (r, c) for r in range(rows)
        for c in range(cols)
        if room[r][c] == "Dirty"
    )

    directions = {
        'Up': (-1, 0),
        'Down': (1, 0),
        'Left': (0, -1),
        'Right': (0, 1)
    }

    # (position, dirty_set, path, depth)
    stack = [(start_pos, initial_dirty, [], 0)]
    visited = set()
    while stack:
        pos, dirty, path, depth = stack.pop()

        state = (pos, tuple(sorted(dirty)), depth)
        if state in visited:
            continue
        visited.add(state)

        # If all clean
        if not dirty:
            return path

        # If depth limit reached
        if depth == limit:
            continue

        # Suck
        if pos in dirty:
            new_dirty = dirty.copy()
            new_dirty.remove(pos)
            stack.append((pos, new_dirty, path + ['Suck'], depth + 1))

        # Move
        for action, (dr, dc) in directions.items():
            new_r, new_c = pos[0] + dr, pos[1] + dc
            if 0 <= new_r < rows and 0 <= new_c < cols:
                new_pos = (new_r, new_c)
                stack.append((new_pos, dirty.copy(), path + [action], depth + 1))

    return None

def ids_vacuum(room, start_pos=(0, 0), max_depth=50):
    for depth in range(max_depth):
        result = dls_vacuum(room, start_pos, depth)
        if result is not None:
            return result
    return None

# 2. DFS
def dfs_vacuum(room, start_pos=(0, 0)):
    rows = len(room)
    cols = len(room[0])

    initial_dirty = set((r, c) for r in range(rows) for c in range(cols) if room[r][c] == "Dirty")

    directions = {
        'Up': (-1, 0),
        'Down': (1, 0),
        'Left': (0, -1),
        'Right': (0, 1)
    }

    stack = [(start_pos, initial_dirty.copy(), [])]
    visited = set()

    while stack:
        pos, dirty, path = stack.pop()
        state = (pos, tuple(sorted(dirty)))

        if state in visited:
            continue
        visited.add(state)

        if not dirty:
            return path

        if pos in dirty:
            new_dirty = dirty.copy()
            new_dirty.remove(pos)
            new_path = path + ['Suck']
            stack.append((pos, new_dirty, new_path))

        for action, (dr, dc) in directions.items():
            new_r, new_c = pos[0] + dr, pos[1] + dc
            if 0 <= new_r < rows and 0 <= new_c < cols:
                new_pos = (new_r, new_c)
                new_path = path + [action]
                stack.append((new_pos, dirty.copy(), new_path))

    return None

# 3. BFS
def bfs_vacuum(room, start_pos=(0, 0)):
    rows = len(room)
    cols = len(room[0])

    initial_dirty = {
        (r, c)
        for r in range(rows)
        for c in range(cols)
        if room[r][c] == "Dirty"
    }

    directions = {
        'Up': (-1, 0),
        'Down': (1, 0),
        'Left': (0, -1),
        'Right': (0, 1)
    }

    queue = deque()
    queue.append((start_pos, initial_dirty.copy(), []))

    visited = set()

    while queue:
        pos, dirty, path = queue.popleft()

        state = (pos, tuple(sorted(dirty)))
        if state in visited:
            continue
        visited.add(state)

        if not dirty:
            return path

        if pos in dirty:
            new_dirty = dirty.copy()
            new_dirty.remove(pos)
            queue.append((pos, new_dirty, path + ['Suck']))

        for action, (dr, dc) in directions.items():
            new_r, new_c = pos[0] + dr, pos[1] + dc
            if 0 <= new_r < rows and 0 <= new_c < cols:
                new_pos = (new_r, new_c)
                queue.append((new_pos, dirty.copy(), path + [action]))

    return None

# 4. A* (Modified to return actions instead of positions)
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_path(room, start, goal):
    rows, cols = len(room), len(room[0])

    open_list = []
    heapq.heappush(open_list, (0, start))

    came_from = {}
    g_cost = {start: 0}

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                new_g = g_cost[current] + 1
                if neighbor not in g_cost or new_g < g_cost[neighbor]:
                    g_cost[neighbor] = new_g
                    f_cost = new_g + heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_cost, neighbor))
                    came_from[neighbor] = current

    return None

def positions_to_actions(path):
    if not path:
        return []
    actions = []
    directions = {
        (-1, 0): 'Up',
        (1, 0): 'Down',
        (0, -1): 'Left',
        (0, 1): 'Right'
    }
    for i in range(1, len(path)):
        dr = path[i][0] - path[i-1][0]
        dc = path[i][1] - path[i-1][1]
        actions.append(directions[(dr, dc)])
    return actions

def vacuum_agent_a_star(room, start=(0, 0)):
    current_pos = start
    total_actions = []

    dirty_cells = [(i, j) for i in range(len(room))
                   for j in range(len(room[0]))
                   if room[i][j] == "Dirty"]

    while dirty_cells:
        target = min(dirty_cells, key=lambda x: heuristic(current_pos, x))
        path = a_star_path(room, current_pos, target)
        if path:
            move_actions = positions_to_actions(path)
            total_actions.extend(move_actions[:-1])  # Move to the position before suck if needed
            if len(move_actions) > 0:
                total_actions.append(move_actions[-1])  # Last move to target
            total_actions.append('Suck')
            current_pos = target
            room[target[0]][target[1]] = "Clean"
            dirty_cells.remove(target)

    return total_actions

# 5. UCS
def ucs_vacuum(room, start_pos=(0, 0)):
    rows = len(room)
    cols = len(room[0])

    initial_dirty = frozenset({
        (r, c)
        for r in range(rows)
        for c in range(cols)
        if room[r][c] == "Dirty"
    })

    directions = {
        'Up': (-1, 0),
        'Down': (1, 0),
        'Left': (0, -1),
        'Right': (0, 1)
    }

    pq = []
    entry_count = 0

    heapq.heappush(pq, (0, entry_count, start_pos, initial_dirty, []))

    visited = set()

    while pq:
        cost, _, pos, dirty, path = heapq.heappop(pq)

        state = (pos, dirty)
        if state in visited:
            continue
        visited.add(state)

        if not dirty:
            return path

        if pos in dirty:
            new_dirty = set(dirty)
            new_dirty.remove(pos)
            new_dirty = frozenset(new_dirty)
            entry_count += 1
            heapq.heappush(pq, (cost + 1, entry_count, pos, new_dirty, path + ['Suck']))

        for action, (dr, dc) in directions.items():
            new_r, new_c = pos[0] + dr, pos[1] + dc
            if 0 <= new_r < rows and 0 <= new_c < cols:
                new_pos = (new_r, new_c)
                entry_count += 1
                heapq.heappush(pq, (cost + 1, entry_count, new_pos, dirty, path + [action]))

    return None

# Main Function
def main():
    start_pos = (0, 0)
    algorithms = ['IDS', 'DFS', 'BFS', 'A*', 'UCS']

    for alg in algorithms:
        # Regenerate room for each algorithm to avoid modifications
        room = generate_environment()

        start_time = time.time()

        if alg == 'IDS':
            path = ids_vacuum(room, start_pos)
        elif alg == 'DFS':
            path = dfs_vacuum(room, start_pos)
        elif alg == 'BFS':
            path = bfs_vacuum(room, start_pos)
        elif alg == 'A*':
            path = vacuum_agent_a_star(room, start_pos)
        elif alg == 'UCS':
            path = ucs_vacuum(room, start_pos)

        end_time = time.time()
        execution_time = end_time - start_time

        print(f"Algorithm: {alg}")
        print(f"Path: {path}")
        print(f"Execution Time: {execution_time:.6f} seconds")
        print("-" * 50)

if __name__ == "__main__":
    main()

Algorithm: IDS
Path: ['Right', 'Suck', 'Down', 'Suck', 'Left', 'Down', 'Down', 'Suck', 'Right', 'Right', 'Down', 'Suck', 'Right', 'Up', 'Up', 'Suck', 'Right', 'Up', 'Up', 'Suck']
Execution Time: 0.166207 seconds
--------------------------------------------------
Algorithm: DFS
Path: ['Right', 'Right', 'Right', 'Right', 'Right', 'Down', 'Left', 'Left', 'Left', 'Left', 'Left', 'Down', 'Right', 'Right', 'Right', 'Right', 'Right', 'Down', 'Left', 'Left', 'Left', 'Left', 'Left', 'Down', 'Right', 'Right', 'Suck', 'Right', 'Right', 'Right', 'Up', 'Left', 'Left', 'Left', 'Left', 'Left', 'Up', 'Right', 'Right', 'Right', 'Right', 'Right', 'Up', 'Left', 'Left', 'Left', 'Left', 'Left', 'Up', 'Right', 'Right', 'Right', 'Right', 'Suck', 'Right', 'Down', 'Left', 'Left', 'Left', 'Left', 'Left', 'Down', 'Right', 'Right', 'Right', 'Right', 'Right', 'Down', 'Left', 'Left', 'Left', 'Left', 'Left', 'Suck', 'Right', 'Right', 'Right', 'Right', 'Right', 'Up', 'Left', 'Left', 'Left', 'Left', 'Left', 'Up', 'Rig